In [1]:
%load_ext autoreload
%autoreload 2
import dgl
from collections import defaultdict
from dgl.nn.pytorch.glob import AvgPooling
from dgllife.model import load_pretrained
from dgllife.utils import mol_to_bigraph, PretrainAtomFeaturizer, PretrainBondFeaturizer, AttentiveFPAtomFeaturizer,AttentiveFPBondFeaturizer, CanonicalAtomFeaturizer, CanonicalBondFeaturizer
from rdkit import Chem
from dgllife.model import AttentiveFPPredictor
from dgllife.utils import Meter, EarlyStopping, SMILESToBigraph
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from torch.optim import Adam
import numpy as np
from dgllife.model import load_pretrained
from dgllife.utils import CanonicalAtomFeaturizer, CanonicalBondFeaturizer
import toolsets.torch_utilities as tu
import dgl
from tqdm import tqdm

In [2]:
train_alc = pd.read_csv('mini_dataset/train_alc.csv')
test_alc = pd.read_csv('mini_dataset/test_alc.csv')

In [160]:
def graph_construction_and_featurization(smiles):
    """Construct graphs from SMILES and featurize them

    Parameters
    ----------
    smiles : list of str
        SMILES of molecules for embedding computation

    Returns
    -------
    list of DGLGraph
        List of graphs constructed and featurized
    list of bool
        Indicators for whether the SMILES string can be
        parsed by RDKit
    """
    graphs = []
    success = []
    for smi in smiles:
        try:
            mol = Chem.MolFromSmiles(smi)
            if mol is None:
                success.append(False)
                continue
            g = mol_to_bigraph(mol, add_self_loop=True,
                               node_featurizer=PretrainAtomFeaturizer(),
                               edge_featurizer=PretrainBondFeaturizer(),
                               canonical_atom_order=False)
            graphs.append(g)
            success.append(True)
        except:
            success.append(False)

    return graphs, success

def collate(graphs):
    return dgl.batch(graphs)
def test(model, test_loader, verbose = False):
    model.eval()
    with torch.no_grad():
        y_true = []
        y_pred = []
        for i, (X_batch, y_batch) in enumerate(test_loader):
            X_batch = X_batch.to('cpu')
            y_batch = y_batch.to('cpu')
            output = model(X_batch)
            y_true.append(y_batch)
            y_pred.append(output)
        y_true = torch.cat(y_true, dim=0)
        y_pred = torch.cat(y_pred, dim=0)
    if verbose:
        print(f'the rmse is {get_rmse(y_true.numpy(), y_pred.numpy()):.4f}')
        print(f'the mae is {get_mae(y_true.numpy(), y_pred.numpy()):.4f}')
    return y_true.numpy(), y_pred.numpy()
def train(model, train_loader,epoch, optimizer, criterion, scheduler = None):
    model.train()
    epoch_loss = 0
    epoch_ns = 0
    for i, (X_batch, y_batch) in enumerate(train_loader):
        X_batch = X_batch.to('cpu')
        y_batch = y_batch.to('cpu')
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        optimizer.zero_grad()
        epoch_loss += loss.detach().item()*y_batch.shape[0]
        epoch_ns += y_batch.shape[0]
    print(f'Epoch {epoch}, Loss: {np.sqrt(epoch_loss/epoch_ns) :.4f}')
def get_mae(y_true, y_pred):
    mae = np.median(np.abs(y_pred - y_true))
    return mae
def get_rmse(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_pred - y_true) ** 2))
    return rmse










class AttentionPooling(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionPooling, self).__init__()
        self.hidden_size = hidden_size
        self.attention_weights = nn.Linear(hidden_size, 1)

    def forward(self, last_hidden_state):
        # last_hidden_state shape: (batch_size, sequence_length, hidden_size)
        
        # Compute attention scores
        attn_scores = self.attention_weights(last_hidden_state)  # (batch_size, sequence_length, 1)
        attn_scores = attn_scores.squeeze(-1)  # (batch_size, sequence_length)
        
        # Apply softmax to get attention weights
        attn_weights = F.softmax(attn_scores, dim=1)  # (batch_size, sequence_length)
        
        # Multiply hidden states by the attention weights
        weighted_hidden_state = last_hidden_state * attn_weights.unsqueeze(-1)  # (batch_size, sequence_length, hidden_size)
        
        # Sum the weighted hidden states to get the final representation
        pooled_output = weighted_hidden_state.sum(dim=1)  # (batch_size, hidden_size)
        
        return pooled_output
from transformers import BertTokenizerFast, BertModel
def make_clf_embeddings(smiles, attention_pooling=False, default_pooler = False):
    checkpoint = 'unikei/bert-base-smiles'
    tokenizer = BertTokenizerFast.from_pretrained(checkpoint)
    model = BertModel.from_pretrained(checkpoint)
    
    with torch.no_grad():
        embeddings_all = torch.empty(0)
        for s in tqdm(smiles):
            tokens = tokenizer(s, return_tensors='pt')
            predictions = model(**tokens)
            if attention_pooling == True:
                attention_pooling = AttentionPooling(hidden_size=768)
                embeddings = attention_pooling(predictions.last_hidden_state)
            elif default_pooler == True:
                embeddings = predictions.pooler_output
            else:
                embeddings = predictions[0][:,0]
            embeddings_all = torch.cat((embeddings_all, embeddings[0].unsqueeze(0)), dim=0)
        
    return embeddings_all.detach().numpy()
def make_dataset(embeddings, labels):
    labels = torch.from_numpy(labels.astype(np.float32))
    labels  = labels.view(labels.shape[0],1)
    embeddings = torch.from_numpy(embeddings.astype(np.float32))
    dataset = TensorDataset( embeddings, labels)
    return dataset
def gin_featurizers(smiles):
    """Construct graphs from SMILES and featurize them

    Parameters
    ----------
    smiles : list of str
        SMILES of molecules for embedding computation

    Returns
    -------
    list of DGLGraph
        List of graphs constructed and featurized
    list of bool
        Indicators for whether the SMILES string can be
        parsed by RDKit
    """
    graphs = []
    success = []
    for smi in smiles:
        try:
            mol = Chem.MolFromSmiles(smi)
            if mol is None:
                success.append(False)
                continue
            g = mol_to_bigraph(mol, add_self_loop=True,
                               node_featurizer=PretrainAtomFeaturizer(),
                               edge_featurizer=PretrainBondFeaturizer(),
                               canonical_atom_order=False)
            graphs.append(g)
            success.append(True)
        except:
            success.append(False)

    return graphs, success

def collate(graphs):
    return dgl.batch(graphs)
def make_gin_embedding(smiles, model_name='gin_supervised_contextpred'):
    gin_feats = gin_featurizers(smiles)
    dataset = [gin_feats[0][x] for x in range(len(gin_feats[1])) if gin_feats[1][x] == True]# if there is fail?
    args = {
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'model': model_name,
        'batch_size': 128
    }
    data_loader = DataLoader(dataset, batch_size=args['batch_size'],
                             collate_fn=collate, shuffle=False)
    
    model = load_pretrained(args['model']).to(args['device'])
    model.eval()
    readout = AvgPooling()
    mol_emb = []
    for batch_id, bg in enumerate(data_loader):
        print('Processing batch {:d}/{:d}'.format(batch_id + 1, len(data_loader)))
        bg = bg.to(args['device'])
        nfeats = [bg.ndata.pop('atomic_number').to(args['device']),
                    bg.ndata.pop('chirality_type').to(args['device'])]
        efeats = [bg.edata.pop('bond_type').to(args['device']),
                    bg.edata.pop('bond_direction_type').to(args['device'])]
        with torch.no_grad():
            node_repr = model(bg, nfeats, efeats)
        mol_emb.append(readout(bg, node_repr))
    mol_emb = torch.cat(mol_emb, dim=0).detach().cpu().numpy()
    return mol_emb

In [5]:
from rdkit.Chem import Descriptors

In [90]:
# def rdkit_featurzier(smiles, use_BCUT = False):
#     descriptors_all = []
#     for s in tqdm(smiles):
#         mol = Chem.MolFromSmiles(s)
#         descriptor_single = []
#         for nm,fn in Descriptors._descList:
#             # some of the descriptor fucntions can throw errors if they fail, catch those here:
#             if 'BCUT' in nm and use_BCUT == False:
#                 continue
#             else:
#                 val = fn(mol)
#             if np.isinf(val):
#                 print(s, nm)
#                 return
#             descriptor_single.append(val)
#         descriptors_all.append(descriptor_single)
#     return np.array(descriptors_all)


def rdkit_featurzier(smiles, use_BCUT = False):
    import deepchem as dc
    all_descriptors = {name: func for name, func in Descriptors.descList}
    all_names = list((all_descriptors.keys()))
    if use_BCUT == False:
        all_names = [x for x in all_names if x.startswith('BCUT2D_')== False]
    f2 = dc.feat.RDKitDescriptors(use_bcut2d= use_BCUT)
    feats2 = f2.featurize(smiles)
    return(feats2)
    # f2 = dc.feat.RDKitDescriptors(use_bcut2d= use_BCUT)
    # feats2 = f2.featurize(smiles)
    # return(feats2[0])

In [150]:
train_retip_md = pd.read_csv('mini_dataset/train_alc_descriptors.csv')
test_retip_md = pd.read_csv('mini_dataset/test_alc_descriptors.csv')

In [158]:
train_retip_md_numpy = train_retip_md.drop("rt", axis=1).values
test_retip_md_numpy = test_retip_md.drop("rt", axis=1).values

In [171]:
# rdkit_descriptor = rdkit_featurzier(train_alc['smiles'], use_BCUT=False)
gin_embedding = make_gin_embedding(train_alc['smiles'])
cls_embedding = make_clf_embeddings(train_alc['smiles'])
features_all = np.concatenate((
                                # rdkit_descriptor,
                                train_retip_md_numpy, 
                                gin_embedding, cls_embedding
                                ), axis=1)
train_data = make_dataset(features_all, train_alc['rt'].values)
# rdkit_descriptor = rdkit_featurzier(test_alc['smiles'], use_BCUT=False)
gin_embedding = make_gin_embedding(test_alc['smiles'])
cls_embedding = make_clf_embeddings(test_alc['smiles'])
features_all = np.concatenate((
                                # rdkit_descriptor, 
                                test_retip_md_numpy, 
                                gin_embedding, cls_embedding
                                ), axis=1)
test_data = make_dataset(features_all, test_alc['rt'].values)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

gin_supervised_contextpred_pre_trained.pth:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

Pretrained model loaded
Processing batch 1/11
Processing batch 2/11
Processing batch 3/11
Processing batch 4/11
Processing batch 5/11
Processing batch 6/11
Processing batch 7/11
Processing batch 8/11
Processing batch 9/11
Processing batch 10/11
Processing batch 11/11


100%|██████████| 1287/1287 [00:49<00:00, 25.88it/s]


gin_supervised_contextpred_pre_trained.pth:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

Pretrained model loaded
Processing batch 1/3
Processing batch 2/3
Processing batch 3/3


100%|██████████| 322/322 [00:12<00:00, 26.20it/s]


In [167]:
class FineTunedModel(nn.Module):
    def __init__(self, input_size=300, num_classes=1):
        super(FineTunedModel, self).__init__()
        self.meta_fc = nn.Sequential(
                    nn.Linear(in_features=input_size, out_features=1024),
                    # nn.BatchNorm1d(512),
                    nn.ReLU(),
                    nn.Linear(in_features=1024, out_features=512),
                    # nn.BatchNorm1d(256),
                    nn.ReLU(),
                    nn.Linear(in_features=512, out_features=num_classes),
                )

    def forward(self, x):
        x = self.meta_fc(x)
        return x

In [174]:
model = FineTunedModel(test_data.tensors[0].shape[1], 1)
optimizer = Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()
num_epochs = 500
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=num_epochs)

best_val_loss = float('inf')

for epoch in range(num_epochs):
    # Forward pass and loss
    train(model, train_loader,epoch, optimizer, criterion, scheduler=scheduler)
    test_result = test(model, test_loader)
    test_loss = get_rmse(test_result[0], test_result[1])
    if test_loss < best_val_loss:
        best_val_loss = test_loss
        torch.save(model.state_dict(), 'best_model.pth')
        patience = 20
    else:
        patience -= 1
        if patience == 0:
            print('early stopping triggered')
            break
model.load_state_dict(torch.load('best_model.pth'))
train_result = test(model, train_loader, verbose=True)
test_result = test(model, test_loader, verbose=True)

Epoch 0, Loss: 26060.6113
Epoch 1, Loss: 10366622.2252
Epoch 2, Loss: 447279.3774
Epoch 3, Loss: 740489.7345
Epoch 4, Loss: 808359.1314
Epoch 5, Loss: 1020205.7600
Epoch 6, Loss: 546457.3390
Epoch 7, Loss: 450216.9465
Epoch 8, Loss: 504111.1495
Epoch 9, Loss: 622998.0685
Epoch 10, Loss: 703014.4047
Epoch 11, Loss: 528765.0723
Epoch 12, Loss: 401842.8069
Epoch 13, Loss: 490998.7073
Epoch 14, Loss: 632344.6830
Epoch 15, Loss: 525511.2521
Epoch 16, Loss: 233556.9044
Epoch 17, Loss: 135611.5181
Epoch 18, Loss: 202717.0058
Epoch 19, Loss: 310452.8113
Epoch 20, Loss: 503593.8544
Epoch 21, Loss: 1875361.1944
Epoch 22, Loss: 2766546.7761
Epoch 23, Loss: 1595969.7974
Epoch 24, Loss: 1192356.2094
Epoch 25, Loss: 799468.7590
Epoch 26, Loss: 359062.7437
Epoch 27, Loss: 637151.6425
Epoch 28, Loss: 830489.2283
Epoch 29, Loss: 107620.4224
Epoch 30, Loss: 2789312.7664
Epoch 31, Loss: 8488167.5079
Epoch 32, Loss: 3981049.7845
Epoch 33, Loss: 16320662.7781
Epoch 34, Loss: 7093952.0794
Epoch 35, Loss: 10

In [169]:
train_result = test(model, train_loader, verbose=True)

the rmse is 15.4126
the mae is 6.5254


In [170]:
test_result = test(model, test_loader, verbose=True)

the rmse is 15.9331
the mae is 7.3023


In [344]:
model = FineTunedModel(1068, 1)
optimizer = Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()


best_val_loss = float('inf')
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass and loss
    train(model, train_loader,epoch, optimizer, criterion)
    test_result = test(model, test_loader)
    test_loss = get_rmse(test_result[0], test_result[1])
    if test_loss < best_val_loss:
        best_val_loss = test_loss
        torch.save(model.state_dict(), 'best_model.pth')
        patience = 10
    else:
        patience -= 1
        if patience == 0:
            print('early stopping triggered')
            break
model.load_state_dict(torch.load('best_model.pth'))
train_result = test(model, train_loader, verbose=True)
test_result = test(model, test_loader, verbose=True)

Epoch 0, Loss: 137.0893
Epoch 1, Loss: 127.2398
Epoch 2, Loss: 103.8848
Epoch 3, Loss: 78.9562
Epoch 4, Loss: 71.0809
Epoch 5, Loss: 68.5411
Epoch 6, Loss: 65.9107
Epoch 7, Loss: 63.4931
Epoch 8, Loss: 61.0713
Epoch 9, Loss: 58.8775
Epoch 10, Loss: 56.9683
Epoch 11, Loss: 55.2881
Epoch 12, Loss: 53.9064
Epoch 13, Loss: 52.6679
Epoch 14, Loss: 51.3747
Epoch 15, Loss: 50.2107
Epoch 16, Loss: 48.9567
Epoch 17, Loss: 47.8767
Epoch 18, Loss: 46.7002
Epoch 19, Loss: 45.4988
Epoch 20, Loss: 44.3301
Epoch 21, Loss: 43.0625
Epoch 22, Loss: 41.7812
Epoch 23, Loss: 40.5730
Epoch 24, Loss: 39.3767
Epoch 25, Loss: 38.4582
Epoch 26, Loss: 37.3449
Epoch 27, Loss: 36.3271
Epoch 28, Loss: 35.2775
Epoch 29, Loss: 34.4186
Epoch 30, Loss: 33.4520
Epoch 31, Loss: 32.6798
Epoch 32, Loss: 31.9098
Epoch 33, Loss: 31.2167
Epoch 34, Loss: 30.4721
Epoch 35, Loss: 29.8006
Epoch 36, Loss: 29.1896
Epoch 37, Loss: 28.6136
Epoch 38, Loss: 28.0820
Epoch 39, Loss: 27.4681
Epoch 40, Loss: 27.0260
Epoch 41, Loss: 26.6218

In [319]:
rdkit_descriptor = rdkit_featurzier(test_alc['smiles'], use_BCUT=False)
gin_embedding = make_gin_embedding(test_alc['smiles'])
cls_embedding = make_clf_embeddings(test_alc['smiles'])
features_all = np.concatenate((rdkit_descriptor, gin_embedding, cls_embedding), axis=1)
test_data = make_dataset(features_all, test_alc['rt'].values)

100%|██████████| 322/322 [00:08<00:00, 36.38it/s]


gin_supervised_contextpred_pre_trained.pth:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

Pretrained model loaded
Processing batch 1/3
Processing batch 2/3
Processing batch 3/3


100%|██████████| 322/322 [00:12<00:00, 24.87it/s]


In [326]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [329]:
model = FineTunedModel(1270, 1)
optimizer = Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()


best_val_loss = float('inf')
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass and loss
    train(model, train_loader,epoch, optimizer, criterion)
    test_result = test(model, test_loader)
    test_loss = get_rmse(test_result[0], test_result[1])
    if test_loss < best_val_loss:
        best_val_loss = test_loss
        torch.save(model.state_dict(), 'best_model.pth')
        patience = 10
    else:
        patience -= 1
        if patience == 0:
            print('early stopping triggered')
            break
model.load_state_dict(torch.load('best_model.pth'))
train_result = test(model, train_loader, verbose=True)
test_result = test(model, test_loader, verbose=True)

Epoch 0, Loss: nan


RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [192]:
tt = test(model, train_loader, verbose=True)

the rmse is 42.5045
the mae is 25.1233


In [193]:
tt = test(model, test_loader, verbose=True)

the rmse is 43.4710
the mae is 26.6265


In [3]:
from dgllife.model.model_zoo.attentivefp_predictor import AttentiveFPPredictor

In [4]:
from dgllife.data import MoleculeCSVDataset

In [5]:
train_alc = pd.read_csv('mini_dataset/train_alc.csv')
test_alc = pd.read_csv('mini_dataset/test_alc.csv')

# below is test code for pretrained GIN representation

In [ ]:
def graph_construction_and_featurization(smiles):
    """Construct graphs from SMILES and featurize them

    Parameters
    ----------
    smiles : list of str
        SMILES of molecules for embedding computation

    Returns
    -------
    list of DGLGraph
        List of graphs constructed and featurized
    list of bool
        Indicators for whether the SMILES string can be
        parsed by RDKit
    """
    graphs = []
    success = []
    for smi in smiles:
        try:
            mol = Chem.MolFromSmiles(smi)
            if mol is None:
                success.append(False)
                continue
            g = mol_to_bigraph(mol, add_self_loop=True,
                               node_featurizer=PretrainAtomFeaturizer(),
                               edge_featurizer=PretrainBondFeaturizer(),
                               canonical_atom_order=False)
            graphs.append(g)
            success.append(True)
        except:
            success.append(False)

    return graphs, success

def collate(graphs):
    return dgl.batch(graphs)
def test(model, test_loader):
    model.eval()
    with torch.no_grad():
        y_true = []
        y_pred = []
        for i, (X_batch, y_batch) in enumerate(test_loader):
            X_batch = X_batch.to('cpu')
            y_batch = y_batch.to('cpu')
            output = model(X_batch)
            y_true.append(y_batch)
            y_pred.append(output)
        y_true = torch.cat(y_true, dim=0)
        y_pred = torch.cat(y_pred, dim=0)
    return y_true.numpy(), y_pred.numpy()
def train(model, train_loader,epoch, optimizer, criterion):
    model.train()
    epoch_loss = 0
    epoch_ns = 0
    for i, (X_batch, y_batch) in enumerate(train_loader):
        X_batch = X_batch.to('cpu')
        y_batch = y_batch.to('cpu')
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.detach().item()*y_batch.shape[0]
        epoch_ns += y_batch.shape[0]
    print(f'Epoch {epoch}, Loss: {np.sqrt(epoch_loss/epoch_ns) :.4f}')
class FineTunedModel(nn.Module):
    def __init__(self, embedding_dim=300, num_classes=1):
        super(FineTunedModel, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.output = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.output(x)
        return x

In [93]:
train_dataset, success = graph_construction_and_featurization(train_alc['smiles'])
test_dataset, test_success = graph_construction_and_featurization(test_alc['smiles'])

In [183]:
train_preloader = DataLoader(train_dataset, batch_size=args['batch_size'],
                             collate_fn=collate, shuffle=False)
test_preloader = DataLoader(test_dataset, batch_size=args['batch_size'],
                             collate_fn=collate, shuffle=False)

In [228]:
args = {
    'batch_size': 64,
    'device': 'cpu',
    'model': 'gin_supervised_masking',
}
model = load_pretrained(args['model']).to(args['device'])
train_df = make_pretrained_datasets(model,train_preloader, train_alc)
test_df = make_pretrained_datasets(model,test_preloader, test_alc)
train_loader = DataLoader(train_df, batch_size=args['batch_size'], shuffle=True)
test_loader = DataLoader(test_df, batch_size=args['batch_size'], shuffle=False)
ft_model = FineTunedModel()
learning_rate = 0.001
# step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(ft_model.parameters(), lr=learning_rate)  
optimizer = torch.optim.Adam(ft_model.parameters(), lr=0.001, 
                            #  weight_decay=1e-4
                             )
best_val_loss = float('inf')
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass and loss
    train(ft_model, train_loader,epoch, optimizer, criterion)
    test_result = test(ft_model, test_loader)
    test_loss = calculate_rmse(test_result[0], test_result[1])
    if test_loss < best_val_loss:
        best_val_loss = test_loss
        torch.save(ft_model.state_dict(), 'best_model.pth')
        patience = 10
    else:
        patience -= 1
        if patience == 0:
            print('early stopping triggered')
            break
ft_model.load_state_dict(torch.load('best_model.pth'))
train_result = test(ft_model, train_loader)
test_result = test(ft_model, test_loader)
print(f'the train rmse is {calculate_rmse(train_result[0], train_result[1])}')
print(f'the train mae is {calculate_mae(train_result[0], train_result[1])}')
print(f'the test rmse is {calculate_rmse(test_result[0], test_result[1])}')
print(f'the test mae is {calculate_mae(test_result[0], test_result[1])}')

gin_supervised_masking_pre_trained.pth:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

Pretrained model loaded


21it [00:01, 18.15it/s]
6it [00:00, 19.74it/s]


Epoch 0, Loss: 137.5318
Epoch 1, Loss: 121.5152
Epoch 2, Loss: 66.6739
Epoch 3, Loss: 48.7676
Epoch 4, Loss: 44.2514
Epoch 5, Loss: 42.5255
Epoch 6, Loss: 41.1153
Epoch 7, Loss: 39.9270
Epoch 8, Loss: 38.7852
Epoch 9, Loss: 37.6807
Epoch 10, Loss: 36.9888
Epoch 11, Loss: 35.6272
Epoch 12, Loss: 34.7508
Epoch 13, Loss: 33.9030
Epoch 14, Loss: 33.0634
Epoch 15, Loss: 32.4630
Epoch 16, Loss: 31.7320
Epoch 17, Loss: 30.9437
Epoch 18, Loss: 30.4970
Epoch 19, Loss: 29.7131
Epoch 20, Loss: 29.0241
Epoch 21, Loss: 28.5882
Epoch 22, Loss: 28.0253
Epoch 23, Loss: 27.3701
Epoch 24, Loss: 26.8970
Epoch 25, Loss: 26.8466
Epoch 26, Loss: 26.2393
Epoch 27, Loss: 25.5648
Epoch 28, Loss: 25.1493
Epoch 29, Loss: 24.8814
Epoch 30, Loss: 24.1900
Epoch 31, Loss: 23.8637
Epoch 32, Loss: 23.6587
Epoch 33, Loss: 23.2250
Epoch 34, Loss: 23.3641
Epoch 35, Loss: 22.6768
Epoch 36, Loss: 22.3339
Epoch 37, Loss: 22.0280
Epoch 38, Loss: 21.8413
Epoch 39, Loss: 21.4737
Epoch 40, Loss: 21.4144
Epoch 41, Loss: 21.1396


# lets load some pretrained models

# this is attentivefp implementation in dgl

In [ ]:
model = load_pretrained('AttentiveFP_Aromaticity') # Pretrained model loaded



for param in model.parameters():
    param.requires_grad = False
model.predict = nn.Sequential(
  nn.Dropout(p=0.2, inplace=False),
  nn.Linear(in_features=200, out_features=1, bias=True)
)
model = model.to('cpu')
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, 
                            #  weight_decay=1e-4
                             )


AttentiveFP_Aromaticity_pre_trained.pth:   0%|          | 0.00/4.59M [00:00<?, ?B/s]

Pretrained model loaded


# configure data

In [15]:
atom_feat = AttentiveFPAtomFeaturizer()
bond_feat = AttentiveFPBondFeaturizer()

def load_dataset(df, atom_feat, bond_feat):
    smiles_to_g = SMILESToBigraph(add_self_loop=False, node_featurizer=atom_feat,
                                  edge_featurizer=bond_feat)
    dataset = MoleculeCSVDataset(df=df,
                                 smiles_to_graph=smiles_to_g,
                                 smiles_column='smiles',
                                 cache_file_path= 'models/graph.bin',
                                 task_names='rt',
                                 n_jobs=6)

    return dataset

In [17]:
train_dataset = load_dataset(train_alc, atom_feat, bond_feat)

test_dataset = load_dataset(test_alc, atom_feat, bond_feat)

Processing dgl graphs from scratch...


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done 100 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 1276 out of 1287 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=6)]: Done 1287 out of 1287 | elapsed:    0.8s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


Processing dgl graphs from scratch...


[Parallel(n_jobs=6)]: Done 100 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 322 out of 322 | elapsed:    0.2s finished


In [22]:
def collate_molgraphs(data):
    """Batching a list of datapoints for dataloader.

    Parameters
    ----------
    data : list of 4-tuples.
        Each tuple is for a single datapoint, consisting of
        a SMILES, a DGLGraph, all-task labels and a binary
        mask indicating the existence of labels.

    Returns
    -------
    smiles : list
        List of smiles
    bg : DGLGraph
        The batched DGLGraph.
    labels : Tensor of dtype float32 and shape (B, T)
        Batched datapoint labels. B is len(data) and
        T is the number of total tasks.
    masks : Tensor of dtype float32 and shape (B, T)
        Batched datapoint binary mask, indicating the
        existence of labels.
    """
    smiles, graphs, labels, masks = map(list, zip(*data))

    bg = dgl.batch(graphs)
    bg.set_n_initializer(dgl.init.zero_initializer)
    bg.set_e_initializer(dgl.init.zero_initializer)
    labels = torch.stack(labels, dim=0)

    if masks is None:
        masks = torch.ones(labels.shape)
    else:
        masks = torch.stack(masks, dim=0)

    return smiles, bg, labels, masks

In [52]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True,
                              collate_fn=collate_molgraphs)
test_loader = DataLoader(dataset=test_dataset, batch_size=64,shuffle=False,
                            collate_fn=collate_molgraphs)

In [40]:
for batch_id, batch_data in enumerate(train_loader):
    break

In [42]:
smiles, bg, labels, masks=batch_data

In [46]:
bg

Graph(num_nodes=3074, num_edges=6068,
      ndata_schemes={'h': Scheme(shape=(39,), dtype=torch.float32)}
      edata_schemes={'e': Scheme(shape=(10,), dtype=torch.float32)})

In [85]:
def predict( model, bg):
    bg = bg.to('cpu')
    
    
    node_feats = bg.ndata.pop('h').to('cpu')
    edge_feats = bg.edata.pop('e').to('cpu')
        # return model(bg, node_feats, edge_feats)
    return model(bg, node_feats, edge_feats)
def run_a_train_epoch( epoch, model, data_loader, loss_criterion, optimizer, scheduler = None):
    model.train()
    # train_meter = Meter()
    total_loss = 0
    total_items = 0
    for batch_id, batch_data in enumerate(data_loader):
        batch_n = len(batch_data[0])
        batch_loss = 0
        smiles, bg, labels, masks = batch_data
        if len(smiles) == 1:
            # Avoid potential issues with batch normalization
            continue

        labels, masks = labels.to('cpu'), masks.to('cpu')
        prediction = predict( model, bg)
        loss = (loss_criterion(prediction, labels) )
        total_loss += loss.item()*batch_n
        total_items += batch_n
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
    median_loss = total_loss / total_items
    print(f'the loss is {np.sqrt(median_loss)}')
def run_an_eval_epoch(model, data_loader, loss_criterion, mode = 'Attentive_FP'):
    model.eval()
    losses = []
    predictions = []
    refs = []
    with torch.no_grad():
        for batch_id, batch_data in enumerate(data_loader):
            smiles, bg, labels, masks = batch_data
            refs.extend(labels)

            if torch.cuda.is_available():
                bg.to(torch.device('cpu'))
                labels = labels.to('cpu')
                masks = masks.to('cpu')
            if mode == 'GAT' or mode == 'GCN':
                prediction = model(bg, bg.ndata['h'])# this is GAT, only node feature required
            else:
                prediction = model(bg, bg.ndata['h'], bg.edata['e'])
            predictions.extend(prediction)
            loss = (loss_criterion(prediction, labels).float())
            #loss = loss_criterion(prediction, labels)
            losses.append(loss.data.item())
    predictions= np.array([i.item() for i in predictions])
    refs = np.array([i.item() for i in refs])
    mse = loss_criterion(torch.tensor(predictions), torch.tensor(refs))
    mae = np.median(abs(predictions-refs))
    return np.sqrt(mse)

In [74]:
from torch.optim import lr_scheduler

In [91]:
model = load_pretrained('AttentiveFP_Aromaticity') # Pretrained model loaded



for param in model.parameters():
    param.requires_grad = False
model.predict = nn.Sequential(
  nn.Linear(in_features=200, out_features=512,bias=True),
  nn.ReLU(),


  nn.Linear(in_features=512, out_features=1, bias=True)
)
model = model.to('cpu')
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, 
                            #  weight_decay=1e-4
                             )
max_epoch = 500
best_val_loss = float('inf')
# step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
for epoch in range(max_epoch):
    
    run_a_train_epoch(epoch, model, train_loader, criterion, optimizer)
    val_rmse = run_an_eval_epoch(model, test_loader, criterion)
    print(f'epoch {epoch}, val_rmse {val_rmse}')
    if val_rmse < best_val_loss:
        best_val_loss = val_rmse
        torch.save(model.state_dict(), 'best_pretrained.pth')
        patience = 10
    else:
        patience -= 1
        if patience == 0:
            print('early stopping triggered')
            break

AttentiveFP_Aromaticity_pre_trained.pth:   0%|          | 0.00/4.59M [00:00<?, ?B/s]

Pretrained model loaded


/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([7])) that is different to the input size (torch.Size([7, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to inc

the loss is 134.29402565122504
epoch 0, val_rmse 130.82691299024117
the loss is 114.9535773867753
epoch 1, val_rmse 100.04459448023646
the loss is 85.38903845279766
epoch 2, val_rmse 74.03280613766817
the loss is 76.13608845337625
epoch 3, val_rmse 74.56561863367233
the loss is 76.04307481702618
epoch 4, val_rmse 76.04688529704741
the loss is 75.83854771989255
epoch 5, val_rmse 75.61223313218494
the loss is 75.89030984020361
epoch 6, val_rmse 75.44847988426986
the loss is 75.81546521506517
epoch 7, val_rmse 75.38103272030811
the loss is 75.85113120573428
epoch 8, val_rmse 75.25285650868307
the loss is 75.8690601282864
epoch 9, val_rmse 74.97371120470247
the loss is 75.83100844906646
epoch 10, val_rmse 75.33707420944404
the loss is 75.86019742180684
epoch 11, val_rmse 74.7128143595593
the loss is 75.94596913346864
epoch 12, val_rmse 74.74965593365646
early stopping triggered


In [70]:
max_epoch = 500
best_val_loss = float('inf')
for epoch in range(max_epoch):
    
    run_a_train_epoch(epoch, model, train_loader, criterion, optimizer)
    val_rmse = run_an_eval_epoch(model, test_loader, criterion)
    print(f'epoch {epoch}, val_rmse {val_rmse}')
    if val_rmse < best_val_loss:
        best_val_loss = val_rmse
        torch.save(model.state_dict(), 'best_model.pth')
        patience = 10
    else:
        patience -= 1
        if patience == 0:
            print('early stopping triggered')
            break

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([7])) that is different to the input size (torch.Size([7, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to inc

the loss is 19180.439416703088
epoch 0, val_rmse 142.6000648394769
the loss is 18831.88536658654
epoch 1, val_rmse 141.26483822911078
the loss is 18488.862283441384
epoch 2, val_rmse 139.9487519517861
the loss is 18154.909683144546
epoch 3, val_rmse 138.6516341838341
the loss is 17837.873118960033
epoch 4, val_rmse 137.40029579893522
the loss is 17514.79140564297
epoch 5, val_rmse 136.141143988239
the loss is 17199.248628108002
epoch 6, val_rmse 134.85920208219378
the loss is 16886.659329169095
epoch 7, val_rmse 133.5840082559893
the loss is 16587.514324434247
epoch 8, val_rmse 132.33844976825915
the loss is 16295.920103984557
epoch 9, val_rmse 131.13530411652042
the loss is 16002.182224893162
epoch 10, val_rmse 129.88643915043139
the loss is 15710.008053795164
epoch 11, val_rmse 128.70046735102142
the loss is 15436.68330237471
epoch 12, val_rmse 127.51026688603586
the loss is 15172.164575441919
epoch 13, val_rmse 126.34515688619975
the loss is 14896.167782087705
epoch 14, val_rmse 125

KeyboardInterrupt: 

In [61]:
train_df = load_dataset(train_alc)
test_df = load_dataset(test_alc)

Processing dgl graphs from scratch...


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done 100 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 1276 out of 1287 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=6)]: Done 1287 out of 1287 | elapsed:    1.0s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


Processing dgl graphs from scratch...


[Parallel(n_jobs=6)]: Done 100 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 322 out of 322 | elapsed:    0.3s finished


In [62]:
Model_name = 'AttentiveFPPredictor'
exp_config = du.get_configure(Model_name)
exp_config['model']=Model_name
in_node_features = CanonicalAtomFeaturizer().feat_size()
in_edge_features = CanonicalBondFeaturizer(self_loop=True).feat_size()
exp_config['in_node_feats']=in_node_features
exp_config['in_edge_feats']=in_edge_features
exp_config['n_tasks'] = 1
args = {
    'device':'cpu',
    'node_featurizer_type':'CanonicalAtomFeaturizer',
    'edge_featurizer':'CanonicalBondFeaturizer',
    'bond_featurizer_type':'CanonicalBondFeaturizer',
    'num_epochs':100,
    'metric':'rmse'
}

In [74]:
Model_name = 'GAT'
exp_config = du.get_configure(Model_name)
exp_config['model']=Model_name
in_node_features = CanonicalAtomFeaturizer().feat_size()
in_edge_features = CanonicalBondFeaturizer(self_loop=True).feat_size()
exp_config['in_node_feats']=in_node_features
# exp_config['in_edge_feats']=in_edge_features
exp_config['n_tasks'] = 1
args = {
    'device':'cpu',
    'node_featurizer_type':'CanonicalAtomFeaturizer',
    'edge_featurizer':'CanonicalBondFeaturizer',
    'bond_featurizer_type':'CanonicalBondFeaturizer',
    'num_epochs':100,
    'metric':'rmse'
}

In [75]:
train_loader = DataLoader(dataset=train_df, batch_size=64, shuffle=True,
                              collate_fn=du.collate_molgraphs, drop_last=True)
test_loader = DataLoader(dataset=test_df, batch_size=64, shuffle=False,
                              collate_fn=du.collate_molgraphs, drop_last=False)

In [98]:
Model_name = 'GCN'
exp_config = du.get_configure(Model_name)
exp_config['model']=Model_name
in_node_features = CanonicalAtomFeaturizer().feat_size()
in_edge_features = CanonicalBondFeaturizer(self_loop=True).feat_size()
exp_config['in_node_feats']=in_node_features
exp_config['in_edge_feats']=in_edge_features
exp_config['n_tasks'] = 1
args = {
    'device':'cpu',
    'node_featurizer_type':'CanonicalAtomFeaturizer',
    'edge_featurizer':'CanonicalBondFeaturizer',
    'bond_featurizer_type':'CanonicalBondFeaturizer',
    'num_epochs':100,
    'metric':'rmse'
}



model = du.load_model(exp_config).to('cpu')
loss_criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), 
                #  lr=exp_config['lr'],
                lr = 0.001,
                     weight_decay=exp_config['weight_decay'])
stopper = EarlyStopping(patience=exp_config['patience'],
                        filename='models' + '/model.pth',
                        metric='rmse')

For metric rmse, the lower the better


In [101]:
for epoch in range(args['num_epochs']):
        # Train
        run_a_train_epoch(args, epoch, model, train_loader, loss_criterion, optimizer, mode='GAT')

        # Validation and early stop
        # val_score = run_an_eval_epoch(args, model, val_loader)
        # early_stop = stopper.step(val_score, model)
        # print('epoch {:d}/{:d}, validation {} {:.4f}, best validation {} {:.4f}'.format(
        #     epoch + 1, args['num_epochs'], args['metric'], val_score,
        #     args['metric'], stopper.best_score))

        # if early_stop:
        #     break

epoch 1/100, training rmse 137.3330
epoch 2/100, training rmse 135.4787
epoch 3/100, training rmse 133.8532
epoch 4/100, training rmse 131.9603
epoch 5/100, training rmse 130.1400
epoch 6/100, training rmse 127.9735
epoch 7/100, training rmse 125.8501
epoch 8/100, training rmse 122.8770
epoch 9/100, training rmse 120.4410
epoch 10/100, training rmse 117.6438
epoch 11/100, training rmse 114.9163
epoch 12/100, training rmse 111.5598
epoch 13/100, training rmse 108.8347
epoch 14/100, training rmse 105.2403
epoch 15/100, training rmse 102.1658
epoch 16/100, training rmse 99.0732
epoch 17/100, training rmse 95.5528
epoch 18/100, training rmse 92.1718
epoch 19/100, training rmse 88.9633
epoch 20/100, training rmse 85.9761
epoch 21/100, training rmse 82.2844
epoch 22/100, training rmse 78.6935
epoch 23/100, training rmse 76.5099
epoch 24/100, training rmse 73.1654
epoch 25/100, training rmse 69.9619
epoch 26/100, training rmse 67.3938
epoch 27/100, training rmse 63.8981
epoch 28/100, training

In [ ]:
# def make_afp_data(data, smiles_col = 'smiles', target_col = 'rt'):

#     train_mols = [Chem.MolFromSmiles(s) for s in data[smiles_col]]
#     train_graph = [mol_to_bigraph(mol,
#                                 node_featurizer=AttentiveFPAtomFeaturizer(),
#                                 edge_featurizer=AttentiveFPBondFeaturizer()) for mol in train_mols]
#     train_y = torch.tensor(data[target_col].values, dtype=torch.float32).reshape(-1, 1)
#     train_smiles = train_alc[smiles_col].values
#     train_data = list(zip(train_smiles, train_graph, train_y))
#     return train_data

In [ ]:
# train_data = make_afp_data(train_alc)
# test_data = make_afp_data(test_alc)

In [104]:
result = run_an_eval_epoch(model, test_loader, loss_criterion, mode = 'GCN')
print(f'the rmse is {result[0]}, the mae is {result[1]}')

the rmse is 15.000260697213859, the mae is 7.7477264404296875


In [103]:
result = run_an_eval_epoch(model, train_loader, loss_criterion, mode = 'GCN')
print(f'the rmse is {result[0]}, the mae is {result[1]}')

the rmse is 11.735922605798882, the mae is 6.880561828613281


In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=64, collate_fn=collate_molgraphs)
test_loader = DataLoader(dataset=test_data, batch_size=64, collate_fn=collate_molgraphs, shuffle=False)

In [ ]:
model = AttentiveFPPredictor(node_feat_size=39,
                                  edge_feat_size=10,
                                  num_layers=2,
                                  num_timesteps=2,
                                  graph_feat_size=256,
                                  n_tasks=1,
                                  dropout=0.0)
model = model.to('cpu')
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0003126662000605776,)
n_epochs = 100
epochs = []
scores = []
for e in range(n_epochs):
    score = run_a_train_epoch(n_epochs, e, model, train_loader, loss_fn, optimizer)
    epochs.append(e)
    scores.append(score)

epoch 1/100, training rmse: 132.1964
epoch 2/100, training rmse: 108.2541
epoch 3/100, training rmse: 59.1010
epoch 4/100, training rmse: 45.2814
epoch 5/100, training rmse: 43.4932
epoch 6/100, training rmse: 42.5811
epoch 7/100, training rmse: 41.5329
epoch 8/100, training rmse: 40.4257
epoch 9/100, training rmse: 39.1514
epoch 10/100, training rmse: 37.1230
epoch 11/100, training rmse: 34.5917
epoch 12/100, training rmse: 31.7194
epoch 13/100, training rmse: 28.7641
epoch 14/100, training rmse: 27.3798
epoch 15/100, training rmse: 26.0786
epoch 16/100, training rmse: 25.0581
epoch 17/100, training rmse: 23.9964
epoch 18/100, training rmse: 23.3594
epoch 19/100, training rmse: 22.8261
epoch 20/100, training rmse: 22.6727
epoch 21/100, training rmse: 22.6882
epoch 22/100, training rmse: 22.2658
epoch 23/100, training rmse: 21.8008
epoch 24/100, training rmse: 21.1478
epoch 25/100, training rmse: 20.5799
epoch 26/100, training rmse: 20.2989
epoch 27/100, training rmse: 20.1153
epoch 28

In [ ]:
rmse, mae, preds, refs = run_an_eval_epoch(model, train_loader, loss_fn)
print(f'training RMSE: {rmse}, MAE: {mae}')

training RMSE: 14.62560367150402, MAE: 5.33612060546875
